#Using PuLP: Coding decision variable, Optimization function ,constraints and solving for optimal solution.

In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/DetailedGeneratedPrices.xlsx'
data = pd.ExcelFile(file_path)
data.sheet_names

['Hotel_Prices', 'Flight_Prices']

In [ ]:
hotel_prices = pd.read_excel(data, 'Hotel_Prices')
flight_prices = pd.read_excel(data, 'Flight_Prices')

hotel_prices.head()

,Date,Hotel,Venue,Hotel_Price
0,2024-05-24,Taj,Delhi-Arun Jaitley Stadium,7.142295
1,2024-05-24,Taj,Mumbai-Wankhede Stadium,9.752226
2,2024-05-24,Taj,Hyderabad-Rajiv Gandhi International Cricket S...,8.612765
3,2024-05-24,Taj,Chennai-M.A. Chidambaram Chepauk Stadium,7.511268
4,2024-05-24,Taj,Kolkata-Eden Gardens,8.153738


In [ ]:
flight_prices.head()

,Date,Source,Destination,Flight_Price
0,2024-05-24,Delhi-Arun Jaitley Stadium,Mumbai-Wankhede Stadium,4.327800
1,2024-05-24,Delhi-Arun Jaitley Stadium,Hyderabad-Rajiv Gandhi International Cricket S...,4.948032
2,2024-05-24,Delhi-Arun Jaitley Stadium,Chennai-M.A. Chidambaram Chepauk Stadium,4.798045
3,2024-05-24,Delhi-Arun Jaitley Stadium,Kolkata-Eden Gardens,4.252076
4,2024-05-24,Delhi-Arun Jaitley Stadium,Mohali- Cricket Association's Bindra Stadium,6.207126


In [ ]:
!pip install pulp


#Coding Formulas

In [ ]:
import pandas as pd
import pulp as pl

hotel_prices['FormattedDate'] = pd.to_datetime(hotel_prices['Date']).dt.date
flight_prices['FormattedDate'] = pd.to_datetime(flight_prices['Date']).dt.date

# Initialize the LP problem
model = pl.LpProblem("IPL_Schedule_Optimization", pl.LpMinimize)

# Define sets
teams = ['Delhi Capitals', 'Kolkata Knight Riders', 'Mumbai Indians', 'Rajasthan Royals',
         'Royal Challengers Bengaluru', 'Sunrisers Hyderabad', 'Chennai Super Kings', 'Punjab Kings']
venues = hotel_prices['Venue'].unique().tolist()
dates = hotel_prices['FormattedDate'].unique().tolist()
hotels = hotel_prices['Hotel'].unique().tolist()
flights = list(zip(flight_prices['Source'], flight_prices['Destination']))

# Create decision variables
hotel_vars = pl.LpVariable.dicts("HotelBooking",
                                 [(team, venue, hotel, date) for team in teams for venue in venues for hotel in hotels for date in dates],
                                 cat='Binary')
flight_vars = pl.LpVariable.dicts("FlightBooking",
                                  [(team, source, destination, date) for team in teams for (source, destination) in flights for date in dates],
                                  cat='Binary')

# Create parameters
hotel_costs = {(hotel, venue, date): price * 3
               for _, (date, hotel, venue, price) in hotel_prices[['FormattedDate', 'Hotel', 'Venue', 'Hotel_Price']].iterrows()}
flight_costs = {(source, destination, date): price
                for _, (date, source, destination, price) in flight_prices[['FormattedDate', 'Source', 'Destination', 'Flight_Price']].iterrows()}

# Objective function
model += pl.lpSum([hotel_costs[hotel, venue, date] * hotel_vars[team, venue, hotel, date]
                   for team in teams for venue in venues for hotel in hotels for date in dates]
                  + [flight_costs[source, destination, date] * flight_vars[team, source, destination, date]
                     for team in teams for (source, destination) in flights for date in dates]), "Total_Cost"

# Constraints
# Travel and match constraints

for team in teams:
    for date in dates:
        if (date + pd.Timedelta(days=1)) in dates:  # Check if the next day is within the range
            for venue in venues:
                # Ensure keys exist before attempting to access them
                if all([
                    (team, source, venue, date) in flight_vars for source in venues
                    if (source, venue) in flights
                ]) and (team, venue, date + pd.Timedelta(days=1)) in hotel_vars:
                    # Teams must travel one day prior to the match
                    model += pl.lpSum(
                        flight_vars[team, source, venue, date] for source in venues
                        if (source, venue) in flights
                    ) == hotel_vars[team, venue, date + pd.Timedelta(days=1)]


# Match scheduling constraints
for team in teams:
    for date in dates:
        # Check if hotel variables exist for the given date before summing
        if all([(team, venue, date) in hotel_vars for venue in venues]):
            # A team plays at most one match per day
            model += pl.lpSum(hotel_vars[team, venue, date] for venue in venues) <= 1
    # Ensure all variables for the total matches constraint exist
    if all([(team, venue, date) in hotel_vars for venue in venues for date in dates]):
        # Each team plays exactly 14 matches
        model += pl.lpSum(hotel_vars[team, venue, date] for venue in venues for date in dates) == 14

# Flight constraints
for team in teams:
    # Filter and sum only valid flights to ensure no key errors
    valid_flights = [
        flight_vars[team, source, destination, date]
        for source in venues
        for destination in venues
        for date in dates
        if (source, destination) in flights and (team, source, destination, date) in flight_vars
    ]
    # Teams travel at least 8 and at most 14 flights
    model += 8 <= pl.lpSum(valid_flights) <= 14

#Rest days constraint
for team in teams:
    for date in dates:
        for date_prime in dates:
            if 0 < abs((date - date_prime).days) <= 2:
                # Check if both dates have variables before adding constraint
                for venue in venues:
                    if (team, venue, date) in hotel_vars and (team, venue, date_prime) in hotel_vars:
                        model += hotel_vars[team, venue, date] + hotel_vars[team, venue, date_prime] <= 1


In [ ]:
model.solve()

1

In [ ]:
print("Optimization Status:", pl.LpStatus[model.status])

Optimization Status: Optimal


#Extracting the meaning from decision variables

In [ ]:
if pl.LpStatus[model.status] == 'Optimal':

    hotel_schedule = {}
    flight_schedule = {}

    for var in hotel_vars.keys():
        if pl.value(hotel_vars[var]) == 1:
            print("var",var)
            team, venue, hotel, date = var
            if team not in hotel_schedule:
                hotel_schedule[team] = []
            hotel_schedule[team].append((date, venue, hotel))

    for var in flight_vars:
        if pl.value(flight_vars[var]) == 1:
            team, source, destination, date = var
            if team not in flight_schedule:
                flight_schedule[team] = []
            flight_schedule[team].append((date, source, destination))

    for team in teams:
        print(f"\nSchedule for {team}:")
        print("Hotels:")
        if team in hotel_schedule:
            for entry in sorted(hotel_schedule[team]):
                date, venue, hotel = entry
                print(f"  On {date}, stay at {hotel} for match at {venue}")
        else:
            print("  No hotel bookings")

        print("Flights:")
        if team in flight_schedule:
            for entry in sorted(flight_schedule[team]):
                date, source, destination = entry
                print(f"  On {date}, fly from {source} to {destination}")
        else:
            print("  No flights scheduled")

else:
    print("No optimal solution found.")


Schedule for Delhi Capitals:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Kolkata Knight Riders:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Mumbai Indians:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Rajasthan Royals:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Royal Challengers Bengaluru:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Sunrisers Hyderabad:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Chennai Super Kings:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled

Schedule for Punjab Kings:
Hotels:
  No hotel bookings
Flights:
  No flights scheduled


In [ ]:
for v in model.variables():
    print(v.name, "=", v.varValue)

Streaming output truncated to the last 5000 lines.
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_11)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_12)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_13)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_14)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_15)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_16)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_17)) = 0.0
HotelBooking_('Punjab_Kings',_'Chennai_M.A._Chidambaram_Chepauk_Stadium',_'ITC',_datetime.date(2024,_7,_18)) = 0.0
HotelBooking_('Punjab_Kings',